In [ ]:
from depmapomics.config import *
from depmapomics import tracker
from depmapomics import upload
from depmapomics import mutations

trackerobj = tracker.initTracker()

In [ ]:
wgs_mat = mutations.generateGermlineMatrix(refworkspace=WGSWORKSPACE,
        vcfdir=WGSVCFDIR,
        savedir="output/" + SAMPLESETNAME + "/",
        filename="binary_mutguides_wgs.tsv.gz",
        bed_location=GUIDESBED,)